# 라이브러리 선언

In [1]:
# 행과열을 다루는 라이브러리
import pandas as pd
import numpy as np

# 시각화 라이브러리
import matplotlib.pyplot as plt
## 머신러닝 모델
from sklearn import neighbors, svm, tree, ensemble

# 머신러닝 전처리 라이브러리
from sklearn.preprocessing import MinMaxScaler

# 머신러닝 정확도 지표 라이브러리
from sklearn.metrics import classification_report

# 데이터 불러오기

In [2]:
csDataUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/customer.csv"
csData = pd.read_csv(csDataUrl)
# 1. 타입 통합 및 숫자형컬럼 추가
csData.columns = csData.columns.str.upper()
balScaler = MinMaxScaler()
stockScaler = MinMaxScaler()

## 특성값 스케일링 (BALANCE, STOCK)
targetColumns = ["BALANCE"]
csData["SC_BALANCE"] = balScaler.fit_transform( csData.loc[:, targetColumns ])
targetColumns = ["STOCK"]
csData["SC_STOCK"] = stockScaler.fit_transform( csData.loc[:, targetColumns ])
csData

## 정답지 LABEL ENCODER
csDict = {"normal":0,
          "diamond":1,
          "vip":2}
csData["LE_LABEL"] = csData.LABEL.map(csDict)

In [3]:
# list( csData.select_dtypes(np.number).columns )

# 2. 특성선정 및 데이터 분리

In [4]:
stdCorr = 0.5

In [5]:
corrDf = csData.corr(numeric_only=True)

In [6]:
features = list( corrDf.loc[ ( corrDf.LE_LABEL >= stdCorr ) &
            ( corrDf.LE_LABEL != 1) ].index )

In [7]:
preFix = "SC"

In [8]:
featuresNew = []

In [9]:
for i in range(0, len(features)):
    if features[i].count(preFix) > 0:
        featuresNew.append(features[i])
    else:
        pass

In [10]:
features = featuresNew.copy()
label = ["LABEL"]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
### 데이터분리
trainData, testData = train_test_split( csData, test_size=0.2, random_state=1)
trainingDataFeatures = trainData.loc[ :, features]
trainingDataLabel = trainData.loc[ :, label]
testDataFeatures = testData.loc[ :, features]
testDataLabel = testData.loc[ :, label]
print(trainingDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataFeatures.shape)
print(testDataLabel.shape)

(16000, 2)
(16000, 1)
(4000, 2)
(4000, 1)


# 3. 모델 선언 및 학습

In [13]:
knnModel = neighbors.KNeighborsClassifier(n_neighbors=5)
fittedKnnModel = knnModel.fit( trainingDataFeatures, trainingDataLabel )

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


# 4. 예측

In [14]:
# csData.BALANCE.min()
# csData.STOCK.min()

In [15]:
inBalance = 30000000
inStock = 17500000

In [16]:
knnPredictValue = fittedKnnModel.predict( testDataFeatures )

# 5. 데이터 정리

In [17]:
testDataLabel["KNN_PREDICT"] = knnPredictValue
print( classification_report( testDataLabel.LABEL, testDataLabel.KNN_PREDICT ) )

              precision    recall  f1-score   support

     diamond       1.00      1.00      1.00      2346
      normal       0.99      0.99      0.99      1188
         vip       1.00      0.99      0.99       466

    accuracy                           0.99      4000
   macro avg       1.00      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000

